In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
import transformers
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,TrainingArguments,Trainer

from src.utils.myutils import *
import yaml
from tqdm import tqdm
import logging
import json
import warnings
import random

logging.disable(logging.ERROR)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning) 

CS_DATA_PATH = PATH + '/data/CS/processed/BABE/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

model_name = 'fav-kky/FERNET-C5'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
BATCH_SIZE = 32

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);

In [37]:
def eval_on_babe(MODEL_PATH,to_train):
    scores=[]
    print("Running 10-fold CV on model: ",model_name,"...")
    for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

        token_train = Dataset.from_dict(token_full[train_index])
        token_valid = Dataset.from_dict(token_full[val_index])

        model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
        if MODEL_PATH is not None:
            model.load_state_dict(torch.load(MODEL_PATH))
        model.to(device)
        if to_train:
            trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
            trainer.train()

            #evaluation
        eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
        scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        print(scores[-1])

    return np.mean(scores)

In [5]:
data = load_dataset('csv',data_files = CS_DATA_PATH)['train']
with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [22]:
subj_path = '/home/horyctom/bias-detection-thesis/src/models/trained/subj_balanced.pth'
mb_path = '/home/horyctom/bias-detection-thesis/src/models/trained/mb_balanced.pth'
wikinpov_path = '/home/horyctom/bias-detection-thesis/src/models/trained/wiki_balanced.pth'
all_path = '/home/horyctom/bias-detection-thesis/src/models/trained/all_balanced.pth'
wncs_path = '/home/horyctom/bias-detection-thesis/src/models/trained/wncs_balanced.pth'

In [8]:
training_args = TrainingArguments(
    output_dir = './',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    logging_steps=50,
    disable_tqdm = False,
    save_total_limit=2,
    weight_decay=0.1,
    learning_rate=3e-5)

In [9]:
token_full = preprocess_data(data,tokenizer,'sentence')

## Vanilla

In [28]:
eval_on_babe(None,True)

0.7835047922235006


## SUBJ

In [29]:
eval_on_babe(subj_path,True)

0.7875124708824497


## WIKI

In [30]:
eval_on_babe(wikinpov_path,True)

0.779679482883832


## MB

In [31]:
eval_on_babe(mb_path,True)

0.7702455353097961


## All

In [32]:
eval_on_babe(all_path,True)

0.7877931723291478


## WNCS

In [33]:
eval_on_babe(wncs_path,True)

0.7824775510066212


## Eval all on BABE not fine-tuning

In [38]:
eval_on_babe(subj_path,False)

Running 10-fold CV on model:  fav-kky/FERNET-C5 ...
0.5706637168141593
0.5455947736140095
0.5366336633663366
0.5680827886710239
0.5866801028091351
0.539760348583878
0.5351981144661361
0.525599128540305
0.5473767177272884
0.5866801028091351


0.5542269457401406

In [39]:
eval_on_babe(wikinpov_path,False)

Running 10-fold CV on model:  fav-kky/FERNET-C5 ...
0.6176034943184889
0.6446758133047906
0.6518423019836843
0.6262673796791443
0.6115673597763411
0.6236265826223986
0.6428220979295457
0.6371895886540238
0.6253924192855491
0.6635030198446936


0.634449005739866

In [40]:
eval_on_babe(mb_path,False)

Running 10-fold CV on model:  fav-kky/FERNET-C5 ...
0.49369869628198937
0.49142444484564
0.4768768768768769
0.4378378378378378
0.40071558596447177
0.4570135746606334
0.4222808441558442
0.5284330677242663
0.46691942182258084
0.4555520682945201


0.46307524184646615

In [41]:
eval_on_babe(all_path,False)

Running 10-fold CV on model:  fav-kky/FERNET-C5 ...
0.6254487435181493
0.6453529321696524
0.647203947368421
0.6281898500102732
0.6474358974358975
0.6344802071772104
0.6666529689747278
0.6529334212261041
0.6183976566113367
0.6568785651883448


0.6422974189680117

In [42]:
eval_on_babe(wncs_path,False)

Running 10-fold CV on model:  fav-kky/FERNET-C5 ...
0.6681572455781242
0.7046179011827867
0.7009574165551962
0.6640707483562087
0.6907764328792543
0.6323232323232324
0.650411016283575
0.6691479186592922
0.6452096285668739
0.6717171717171717


0.6697388712101715